In [143]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import math
import gc

from sklearn.metrics import roc_auc_score

import lightgbm as lgbm

import pickle
def dump_pkl(data, filename):
  with open(filename, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_pkl(filename):
  with open(filename, 'rb') as handle:
    data = pickle.load(handle)
  return data

In [145]:
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 50)

In [148]:
df = load_pkl('../data/feat_engin/lgbm/data_15min_1hour_1day.pkl')

In [247]:
model = load_pkl('../bst_mdl/model_+0.5_-1.5_4hour/model_+0.5_-1.5_4hour.pkl')

features_fe = ['close/close_w10_ma',
 'close/close_w20_ma',
 'close/close_w30_ma',
 'close/close_w10_expma',
 'close/close_w20_expma',
 'close/close_w30_expma']

THRESHOLD = 0.16409330252458745

In [249]:
df['y_pred'] = model.predict(df[features_fe])

In [251]:
df['y_pred']

0          0.127518
1          0.149727
             ...   
2362115    0.079785
2362116    0.079785
Name: y_pred, Length: 2362117, dtype: float64

In [255]:

mask_hour = np.array(pd.Timestamp('2025-02-17 16:00:00') == df['time'])
mask_hour.sum()

67

In [257]:
df_hour = df.loc[mask_hour, ['time', 'ticker', 'close', 'y_pred']+features_fe].sort_values(by='ticker').reset_index(drop=True)
df_hour

,time,ticker,close,y_pred,close/close_w10_ma,close/close_w20_ma,close/close_w30_ma,close/close_w10_expma,close/close_w20_expma,close/close_w30_expma
0,2025-02-17 16:00:00,ABIO,92.180000,0.079785,0.998743,0.997522,0.998419,0.998929,0.998769,0.999341
1,2025-02-17 16:00:00,AFKS,16.707001,0.099571,1.006331,1.006494,1.011816,1.004659,1.008418,1.012294
...,...,...,...,...,...,...,...,...,...,...
65,2025-02-17 16:00:00,X5,3466.500000,0.079785,1.002255,1.002980,1.005793,1.001678,1.003705,1.005695
66,2025-02-17 16:00:00,YDEX,4676.000000,0.079785,1.003369,1.006159,1.009848,1.003215,1.006368,1.009298


#### load dev data

In [261]:
#prediction
df_dev_res = load_pkl(f'../dev/data/result/data_result_2025-02-17_16_15.pkl')
df_dev_res = df_dev_res.sort_values(by='ticker')
df_dev_res

,time,ticker,close,model,y_pred,y_pred_bin,fix_lose,fix_win
94,2025-02-17 16:00:00,ABIO,92.18,long_-0.5_+1.5_4hour,0.073012,0,91.7191,93.5627
27,2025-02-17 16:00:00,ABIO,92.18,short_+0.5_-1.5_4hour,0.079785,0,92.6409,90.7973
...,...,...,...,...,...,...,...,...
90,2025-02-17 16:00:00,YDEX,4676.00,long_-0.5_+1.5_4hour,0.137134,0,4652.6200,4746.1400
23,2025-02-17 16:00:00,YDEX,4676.00,short_+0.5_-1.5_4hour,0.079785,0,4699.3800,4605.8600


In [263]:
df_dev_res['model'].unique()

array(['long_-0.5_+1.5_4hour', 'short_+0.5_-1.5_4hour'], dtype=object)

In [265]:
df_dev_res = df_dev_res[df_dev_res['model'] == 'short_+0.5_-1.5_4hour'].reset_index(drop=True)
df_dev_res

,time,ticker,close,model,y_pred,y_pred_bin,fix_lose,fix_win
0,2025-02-17 16:00:00,ABIO,92.180,short_+0.5_-1.5_4hour,0.079785,0,92.640900,90.797300
1,2025-02-17 16:00:00,AFKS,16.707,short_+0.5_-1.5_4hour,0.099571,0,16.790535,16.456395
...,...,...,...,...,...,...,...,...
65,2025-02-17 16:00:00,X5,3466.500,short_+0.5_-1.5_4hour,0.079785,0,3483.832500,3414.502500
66,2025-02-17 16:00:00,YDEX,4676.000,short_+0.5_-1.5_4hour,0.079785,0,4699.380000,4605.860000


In [267]:
#features
df_dev_lgbm = load_pkl(f'../dev/data/lgbm/data_lgbm_2025-02-17_16_15.pkl')
df_dev_lgbm = df_dev_lgbm.sort_values(by='ticker').reset_index(drop=True)
df_dev_lgbm

,time,ticker,close,time_1hour,time_1day,close/close_w10_expma,close/close_w30_expma,close/close_w20_ma,close/close_w10_ma,close_w10_max/close,close/close_w30_ma,close/close_1hour_w10_expma,close/close_w20_expma
0,2025-02-17 16:00:00,ABIO,92.180,2025-02-17 15:00:00,2025-02-14 03:00:00,0.998929,0.999339,0.997522,0.998743,1.004556,0.998419,0.999654,0.998769
1,2025-02-17 16:00:00,AFKS,16.707,2025-02-17 15:00:00,2025-02-14 03:00:00,1.004659,1.012264,1.006494,1.006331,1.000718,1.011816,1.015907,1.008418
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2025-02-17 16:00:00,X5,3466.500,2025-02-17 15:00:00,2025-02-14 03:00:00,1.001678,1.005692,1.002980,1.002255,1.001010,1.005793,1.007157,1.003705
66,2025-02-17 16:00:00,YDEX,4676.000,2025-02-17 15:00:00,2025-02-14 03:00:00,1.003215,1.009285,1.006159,1.003369,1.000000,1.009848,1.012156,1.006368


In [271]:
(df_dev_res['ticker'] == df_dev_lgbm['ticker']).all(), (df_dev_res['time'] == df_dev_lgbm['time']).all()

(True, True)

In [274]:


df_dev = df_dev_res.merge(df_dev_lgbm, on=['time', 'ticker'], how='inner', suffixes=('', '2'))

df_dev

,time,ticker,close,model,y_pred,y_pred_bin,fix_lose,fix_win,close2,time_1hour,time_1day,close/close_w10_expma,close/close_w30_expma,close/close_w20_ma,close/close_w10_ma,close_w10_max/close,close/close_w30_ma,close/close_1hour_w10_expma,close/close_w20_expma
0,2025-02-17 16:00:00,ABIO,92.180,short_+0.5_-1.5_4hour,0.079785,0,92.640900,90.797300,92.180,2025-02-17 15:00:00,2025-02-14 03:00:00,0.998929,0.999339,0.997522,0.998743,1.004556,0.998419,0.999654,0.998769
1,2025-02-17 16:00:00,AFKS,16.707,short_+0.5_-1.5_4hour,0.099571,0,16.790535,16.456395,16.707,2025-02-17 15:00:00,2025-02-14 03:00:00,1.004659,1.012264,1.006494,1.006331,1.000718,1.011816,1.015907,1.008418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2025-02-17 16:00:00,X5,3466.500,short_+0.5_-1.5_4hour,0.079785,0,3483.832500,3414.502500,3466.500,2025-02-17 15:00:00,2025-02-14 03:00:00,1.001678,1.005692,1.002980,1.002255,1.001010,1.005793,1.007157,1.003705
66,2025-02-17 16:00:00,YDEX,4676.000,short_+0.5_-1.5_4hour,0.079785,0,4699.380000,4605.860000,4676.000,2025-02-17 15:00:00,2025-02-14 03:00:00,1.003215,1.009285,1.006159,1.003369,1.000000,1.009848,1.012156,1.006368


In [276]:
del df_dev_lgbm, df_dev_res

In [278]:
(df_dev['close'] == df_dev['close2']).all()

True

In [281]:
(df_dev['ticker'] == df_hour['ticker']).all()

True

In [283]:
df_union = pd.concat([df_dev[['time', 'ticker', 'close', 'y_pred', 'y_pred_bin']].reset_index(drop=True), df_hour[['time', 'ticker', 'close', 'y_pred']].reset_index(drop=True)], axis=1)
df_union

,time,ticker,close,y_pred,y_pred_bin,time,ticker,close,y_pred
0,2025-02-17 16:00:00,ABIO,92.180,0.079785,0,2025-02-17 16:00:00,ABIO,92.180000,0.079785
1,2025-02-17 16:00:00,AFKS,16.707,0.099571,0,2025-02-17 16:00:00,AFKS,16.707001,0.099571
...,...,...,...,...,...,...,...,...,...
65,2025-02-17 16:00:00,X5,3466.500,0.079785,0,2025-02-17 16:00:00,X5,3466.500000,0.079785
66,2025-02-17 16:00:00,YDEX,4676.000,0.079785,0,2025-02-17 16:00:00,YDEX,4676.000000,0.079785


In [286]:
(df_dev['y_pred'].values != df_hour['y_pred'].values).sum(), (df_dev['close'].values != df_hour['close'].values).sum()

(0, 50)

In [289]:
df_union[df_dev['close'].values != df_hour['close'].values]

,time,ticker,close,y_pred,y_pred_bin,time,ticker,close,y_pred
0,2025-02-17 16:00:00,ABIO,92.180,0.079785,0,2025-02-17 16:00:00,ABIO,92.180000,0.079785
1,2025-02-17 16:00:00,AFKS,16.707,0.099571,0,2025-02-17 16:00:00,AFKS,16.707001,0.099571
...,...,...,...,...,...,...,...,...,...
63,2025-02-17 16:00:00,VTBR,93.940,0.090657,0,2025-02-17 16:00:00,VTBR,93.940002,0.090657
64,2025-02-17 16:00:00,WUSH,195.150,0.161469,0,2025-02-17 16:00:00,WUSH,195.149994,0.161469


In [291]:
#close

# > 0.01% = 0
# > 0.0001% = 0
mask = (np.abs(df_dev['close'].values - df_hour['close'].values) / df_hour['close'].values) > 0.00_00_01
df_union.loc[mask]

,time,ticker,close,y_pred,y_pred_bin,time,ticker,close,y_pred


In [293]:
# C close все норм

In [296]:
(df_dev['y_pred'].values != df_hour['y_pred'].values).sum(), (df_dev['y_pred'].values != df_hour['y_pred'].values).mean()

(0, 0.0)

In [298]:
mask = (np.abs(df_dev['y_pred'].values - df_hour['y_pred'].values) / df_hour['y_pred'].values) > 0.00_00_01
print(mask.sum())
df_union.loc[mask]

0


,time,ticker,close,y_pred,y_pred_bin,time,ticker,close,y_pred


In [300]:
#Все отлично

#### features

In [304]:
#по сэмплам
(np.abs(df_dev[features_fe].values - df_hour[features_fe].values) / df_hour[features_fe].values > 0.00_00_01).any(axis=1).mean()

0.9402985074626866

In [306]:
#по сэмплам
(np.abs(df_dev[features_fe].values - df_hour[features_fe].values) / df_hour[features_fe].values > 0.00_01).any(axis=1).mean()

0.0

In [308]:
#по фичам
(np.abs(df_dev[features_fe].values - df_hour[features_fe].values) / df_hour[features_fe].values > 0.00_00_01).any(axis=0).mean()

0.3333333333333333

In [310]:
#по фичам
(np.abs(df_dev[features_fe].values - df_hour[features_fe].values) / df_hour[features_fe].values > 0.00_01).any(axis=0).mean()

0.0

In [312]:
#не четко

In [314]:
list(zip(features_fe, (np.abs(df_dev[features_fe].values - df_hour[features_fe].values) / df_hour[features_fe].values > 0.00_00_01).any(axis=0)))

[('close/close_w10_ma', False),
 ('close/close_w20_ma', False),
 ('close/close_w30_ma', False),
 ('close/close_w10_expma', False),
 ('close/close_w20_expma', True),
 ('close/close_w30_expma', True)]

In [316]:
list(zip(df_dev['ticker'].tolist(), (np.abs(df_dev[features_fe].values - df_hour[features_fe].values) / df_hour[features_fe].values > 0.00_00_01).any(axis=1)))

[('ABIO', True),
 ('AFKS', True),
 ('AFLT', True),
 ('ALRS', True),
 ('BANEP', True),
 ('BELU', True),
 ('BSPB', True),
 ('CHMF', True),
 ('ENPG', True),
 ('FEES', True),
 ('FESH', True),
 ('FLOT', True),
 ('GAZP', True),
 ('GMKN', True),
 ('GTRK', True),
 ('HEAD', True),
 ('HYDR', True),
 ('IRAO', True),
 ('IRKT', True),
 ('LKOH', True),
 ('LSRG', True),
 ('MAGN', True),
 ('MDMG', True),
 ('MGNT', True),
 ('MOEX', True),
 ('MTLR', True),
 ('MTLRP', True),
 ('MTSS', True),
 ('MVID', True),
 ('NLMK', True),
 ('NMTP', True),
 ('NVTK', True),
 ('PHOR', False),
 ('PIKK', True),
 ('PLZL', True),
 ('POSI', True),
 ('RASP', True),
 ('RNFT', True),
 ('ROSN', True),
 ('RTKM', True),
 ('RUAL', True),
 ('SBER', True),
 ('SBERP', True),
 ('SELG', True),
 ('SFIN', True),
 ('SGZH', True),
 ('SIBN', True),
 ('SMLT', True),
 ('SNGS', True),
 ('SNGSP', True),
 ('SPBE', True),
 ('SVAV', False),
 ('T', True),
 ('TATN', True),
 ('TATNP', False),
 ('TGKN', True),
 ('TRMK', True),
 ('TRNFP', True),
 ('UNAC'

In [320]:
df_calc_features = pd.concat([df_dev[['ticker']+features_fe].reset_index(drop=True), df_hour[['ticker']+features_fe].reset_index(drop=True)], axis=1)
df_calc_features

                         

,ticker,close/close_w10_ma,close/close_w20_ma,close/close_w30_ma,close/close_w10_expma,close/close_w20_expma,close/close_w30_expma,ticker,close/close_w10_ma,close/close_w20_ma,close/close_w30_ma,close/close_w10_expma,close/close_w20_expma,close/close_w30_expma
0,ABIO,0.998743,0.997522,0.998419,0.998929,0.998769,0.999339,ABIO,0.998743,0.997522,0.998419,0.998929,0.998769,0.999341
1,AFKS,1.006331,1.006494,1.011816,1.004659,1.008418,1.012264,AFKS,1.006331,1.006494,1.011816,1.004659,1.008418,1.012294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,X5,1.002255,1.002980,1.005793,1.001678,1.003705,1.005692,X5,1.002255,1.002980,1.005793,1.001678,1.003705,1.005695
66,YDEX,1.003369,1.006159,1.009848,1.003215,1.006368,1.009285,YDEX,1.003369,1.006159,1.009848,1.003215,1.006368,1.009298


In [322]:
'close/close_w30_expma'

'close/close_w30_expma'

In [324]:
def calculate_exp_ma(data, window):
    return data.ewm(span=window, min_periods=window).mean().squeeze().values

In [ ]:
#Вывод: expma так считается, но вроде разница незначительна. Но на expema 60 и 120 уже будет большая скорее всего - чем больше окно тем хуже в моей реализации прода

# Сверим в данных

### *1day

In [141]:
ticker = 'YDEX'
df_dev_ticker = load_pkl('../dev/data/preproc/1day/2025-02-18/YDEX.pkl')
df_dev_ticker

,time,close,volume,ticker,tmos_close
0,2024-08-26 03:00:00,3913.5,1751792.0,YDEX,5.93
1,2024-08-27 03:00:00,3844.0,910370.0,YDEX,5.85
...,...,...,...,...,...
119,2025-02-14 03:00:00,4549.0,1680291.0,YDEX,7.02
120,2025-02-17 03:00:00,4709.0,1210212.0,YDEX,7.34


In [148]:
df_1day = load_pkl('../data/feat_engin/df_1day_fe.pkl')
df_1day

,index_1day,time_1day,close_1day,volume_1day,ticker,tmos_close_1day,date_index,close_1day_w1_roc,close_1day_w1_diff,volume_1day_w1_roc,volume_1day_w1_diff,tmos_close_1day_w1_roc,tmos_close_1day_w1_diff,close_1day_w5_ma,close_1day_w5_std,close_1day_w5_norm_std,close_1day_w5_ma_low_2std,close_1day_w5_ma_up_2std,close_1day_w5_ma_low_3std,close_1day_w5_ma_up_3std,close_1day_w5_mean_abs_pct,close_1day_w5_alpha,close_1day_w5_min,close_1day_w5_max,close_1day_w5_rsi,...,close_1day_w120_lvl_-0.99-0.995,close_1day_w120_lvl_1.01-1.02,close_1day_w120_lvl_-0.98-0.99,close_1day_w120_lvl_1.02-1.03,close_1day_w120_lvl_-0.97-0.98,close_1day_w120_lvl_1.03-1.04,close_1day_w120_lvl_-0.96-0.97,close_1day_w120_lvl_1.04-1.05,close_1day_w120_lvl_-0.95-0.96,close_1day_w120_lvl_1.05-1.07,close_1day_w120_lvl_-0.9299999999999999-0.95,tmos_close_1day_w120_lvl_1-1.005,tmos_close_1day_w120_lvl_-0.995-1,tmos_close_1day_w120_lvl_1.005-1.01,tmos_close_1day_w120_lvl_-0.99-0.995,tmos_close_1day_w120_lvl_1.01-1.015,tmos_close_1day_w120_lvl_-0.985-0.99,tmos_close_1day_w120_lvl_1.015-1.02,tmos_close_1day_w120_lvl_-0.98-0.985,tmos_close_1day_w120_lvl_1.02-1.03,tmos_close_1day_w120_lvl_-0.97-0.98,tmos_close_1day_w120_lvl_1.03-1.04,tmos_close_1day_w120_lvl_-0.96-0.97,tmos_close_1day_w120_lvl_1.04-1.05,tmos_close_1day_w120_lvl_-0.95-0.96
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,2022-05-04 03:00:00,31.20,1138130.0,AFLT,4.18,2022-05-04,NaN,NaN,NaN,NaN,NaN,NaN,31.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.20,31.2,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2022-05-05 03:00:00,30.70,664101.0,AFLT,4.22,2022-05-05,-0.016026,-0.50,-0.416498,-474029.0,0.009569,0.04,30.950,0.353553,0.011423,30.242893,31.657107,29.889340,32.010660,0.016026,-2.000000,30.70,31.2,0.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46868,46868,2025-02-14 03:00:00,390.30,246080.0,BSPB,7.02,2025-02-14,-0.010997,-4.34,0.069206,15928.0,0.002857,0.02,390.078,6.903291,0.017697,376.271419,403.884581,369.368128,410.787872,0.015544,0.145262,380.10,398.0,74.333333,...,1.0,3.0,2.0,0.0,5.0,0.0,10.0,0.0,8.0,0.0,17.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46869,46869,2025-02-17 03:00:00,397.56,201930.0,BSPB,7.34,2025-02-17,0.018601,7.26,-0.179413,-44150.0,0.045584,0.32,393.570,4.638728,0.011786,384.292543,402.847457,379.653815,407.486185,0.016922,0.147785,387.35,398.0,76.567255,...,2.0,0.0,4.0,0.0,2.0,0.0,1.0,0.0,5.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [156]:
mask_time = (pd.Timestamp('2024-08-26') < df_1day['time_1day']) & (df_1day['time_1day'] < pd.Timestamp('2025-02-18'))
mask = (df_1day['ticker'] == ticker) & mask_time
df_1day_ticker = df_1day[mask]
df_1day_ticker

,index_1day,time_1day,close_1day,volume_1day,ticker,tmos_close_1day,date_index,close_1day_w1_roc,close_1day_w1_diff,volume_1day_w1_roc,volume_1day_w1_diff,tmos_close_1day_w1_roc,tmos_close_1day_w1_diff,close_1day_w5_ma,close_1day_w5_std,close_1day_w5_norm_std,close_1day_w5_ma_low_2std,close_1day_w5_ma_up_2std,close_1day_w5_ma_low_3std,close_1day_w5_ma_up_3std,close_1day_w5_mean_abs_pct,close_1day_w5_alpha,close_1day_w5_min,close_1day_w5_max,close_1day_w5_rsi,...,close_1day_w120_lvl_-0.99-0.995,close_1day_w120_lvl_1.01-1.02,close_1day_w120_lvl_-0.98-0.99,close_1day_w120_lvl_1.02-1.03,close_1day_w120_lvl_-0.97-0.98,close_1day_w120_lvl_1.03-1.04,close_1day_w120_lvl_-0.96-0.97,close_1day_w120_lvl_1.04-1.05,close_1day_w120_lvl_-0.95-0.96,close_1day_w120_lvl_1.05-1.07,close_1day_w120_lvl_-0.9299999999999999-0.95,tmos_close_1day_w120_lvl_1-1.005,tmos_close_1day_w120_lvl_-0.995-1,tmos_close_1day_w120_lvl_1.005-1.01,tmos_close_1day_w120_lvl_-0.99-0.995,tmos_close_1day_w120_lvl_1.01-1.015,tmos_close_1day_w120_lvl_-0.985-0.99,tmos_close_1day_w120_lvl_1.015-1.02,tmos_close_1day_w120_lvl_-0.98-0.985,tmos_close_1day_w120_lvl_1.02-1.03,tmos_close_1day_w120_lvl_-0.97-0.98,tmos_close_1day_w120_lvl_1.03-1.04,tmos_close_1day_w120_lvl_-0.96-0.97,tmos_close_1day_w120_lvl_1.04-1.05,tmos_close_1day_w120_lvl_-0.95-0.96
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16568,16568,2024-08-26 03:00:00,3913.5,1751792.0,YDEX,5.93,2024-08-26,0.027975,106.5,0.260166,361664.0,0.034904,0.20,3820.8,59.885933,0.015674,3701.028134,3940.571866,3641.142200,4000.457800,0.014422,0.020320,3753.0,3913.5,84.432234,...,2.0,7.0,6.0,7.0,6.0,7.0,2.0,4.0,3.0,11.0,3.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,5.0,1.0,3.0,1.0,2.0,0.0
16569,16569,2024-08-27 03:00:00,3844.0,910370.0,YDEX,5.85,2024-08-27,-0.017759,-69.5,-0.480321,-841422.0,-0.013491,-0.08,3839.0,46.450242,0.012100,3746.099516,3931.900484,3699.649273,3978.350727,0.016497,0.015584,3794.0,3913.5,64.444444,...,1.0,5.0,3.0,5.0,1.0,8.0,3.0,5.0,1.0,11.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,0.0,1.0,1.0,4.0,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16687,16687,2025-02-14 03:00:00,4549.0,1680291.0,YDEX,7.02,2025-02-14,0.012689,57.0,0.008121,13536.0,0.002857,0.02,4412.9,150.309015,0.034061,4112.281970,4713.518030,3961.972955,4863.827045,0.018731,0.009616,4217.0,4549.0,93.193069,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16688,16688,2025-02-17 03:00:00,4709.0,1210212.0,YDEX,7.34,2025-02-17,0.035173,160.0,-0.279760,-470079.0,0.045584,0.32,4511.3,151.043702,0.033481,4209.212595,4813.387405,4058.168893,4964.431107,0.024956,0.009572,4287.0,4709.0,94.972578,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [159]:
#close, volume, tmos_close
df_dev_ticker['close'].values == df_1day_ticker['close_1day'].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [161]:
pd.concat([df_dev_ticker['close'].reset_index(drop=True), df_1day_ticker['close_1day'].reset_index(drop=True)], axis=1)

,close,close_1day
0,3913.5,3913.5
1,3844.0,3844.0
...,...,...
119,4549.0,4549.0
120,4709.0,4709.0


In [165]:
df_dev_ticker['volume'].values == df_1day_ticker['volume_1day'].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [167]:
df_dev_ticker['tmos_close'].values == df_1day_ticker['tmos_close_1day'].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

### *1hour

In [84]:
ticker = 'YDEX'
df_dev_ticker = load_pkl('../dev/data/preproc/1hour/2025-02-17_16/YDEX.pkl')
df_dev_ticker

,time,close,volume,ticker,tmos_close
0,2025-02-04 18:00:00,4021.5,12373.0,YDEX,6.38
1,2025-02-04 19:00:00,4019.0,8899.0,YDEX,6.40
...,...,...,...,...,...
119,2025-02-17 14:00:00,4654.5,41682.0,YDEX,7.15
120,2025-02-17 15:00:00,4675.5,66360.0,YDEX,7.18


In [86]:
df_1day = load_pkl('../data/feat_engin/df_1hour_fe.pkl')
df_1day

,index_1hour,time_1hour,close_1hour,volume_1hour,ticker,tmos_close_1hour,date_hour_index,close_1hour_w1_roc,close_1hour_w1_diff,volume_1hour_w1_roc,volume_1hour_w1_diff,tmos_close_1hour_w1_roc,tmos_close_1hour_w1_diff,close_1hour_w5_ma,close_1hour_w5_std,close_1hour_w5_norm_std,close_1hour_w5_ma_low_2std,close_1hour_w5_ma_up_2std,close_1hour_w5_ma_low_3std,close_1hour_w5_ma_up_3std,close_1hour_w5_mean_abs_pct,close_1hour_w5_alpha,close_1hour_w5_min,close_1hour_w5_max,close_1hour_w5_rsi,...,close_1hour_w120_lvl_-0.99-0.995,close_1hour_w120_lvl_1.01-1.02,close_1hour_w120_lvl_-0.98-0.99,close_1hour_w120_lvl_1.02-1.03,close_1hour_w120_lvl_-0.97-0.98,close_1hour_w120_lvl_1.03-1.04,close_1hour_w120_lvl_-0.96-0.97,close_1hour_w120_lvl_1.04-1.05,close_1hour_w120_lvl_-0.95-0.96,close_1hour_w120_lvl_1.05-1.07,close_1hour_w120_lvl_-0.9299999999999999-0.95,tmos_close_1hour_w120_lvl_1-1.005,tmos_close_1hour_w120_lvl_-0.995-1,tmos_close_1hour_w120_lvl_1.005-1.01,tmos_close_1hour_w120_lvl_-0.99-0.995,tmos_close_1hour_w120_lvl_1.01-1.015,tmos_close_1hour_w120_lvl_-0.985-0.99,tmos_close_1hour_w120_lvl_1.015-1.02,tmos_close_1hour_w120_lvl_-0.98-0.985,tmos_close_1hour_w120_lvl_1.02-1.03,tmos_close_1hour_w120_lvl_-0.97-0.98,tmos_close_1hour_w120_lvl_1.03-1.04,tmos_close_1hour_w120_lvl_-0.96-0.97,tmos_close_1hour_w120_lvl_1.04-1.05,tmos_close_1hour_w120_lvl_-0.95-0.96
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,2022-05-04 10:00:00,32.10,503380.0,AFLT,4.27,2022-05-04 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,32.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.10,32.10,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2022-05-04 11:00:00,31.38,191664.0,AFLT,4.20,2022-05-04 11:00:00,-0.022430,-0.72,-0.619246,-311716.0,-0.016393,-0.07,31.740,0.509117,0.016040,30.721766,32.758234,30.212649,33.267351,0.022430,-1.388889,31.38,32.10,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594230,594230,2025-02-17 22:00:00,396.18,10624.0,BSPB,7.30,2025-02-17 22:00:00,-0.001009,-0.40,1.037591,5410.0,0.005510,0.04,395.846,0.699593,0.001767,394.446815,397.245185,393.747222,397.944778,0.001127,1.956357,394.79,396.58,80.269058,...,8.0,0.0,12.0,0.0,18.0,0.0,14.0,0.0,19.0,0.0,25.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,6.0,0.0,3.0,0.0,12.0,0.0,11.0
594231,594231,2025-02-17 23:00:00,397.56,5673.0,BSPB,7.34,2025-02-17 23:00:00,0.003483,1.38,-0.466020,-4951.0,0.005479,0.04,396.400,0.749300,0.001890,394.901401,397.898599,394.152101,398.647899,0.001804,1.821177,395.53,397.56,88.795518,...,10.0,0.0,18.0,0.0,17.0,0.0,11.0,0.0,17.0,0.0,31.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,2.0,0.0,9.0,0.0,2.0,0.0,18.0


In [92]:
mask_time = (pd.Timestamp('2025-02-04 18:00') <= df_1day['time_1hour']) & (df_1day['time_1hour'] < pd.Timestamp('2025-02-17 16:00'))
mask = (df_1day['ticker'] == ticker) & mask_time
df_1day_ticker = df_1day[mask]
df_1day_ticker

,index_1hour,time_1hour,close_1hour,volume_1hour,ticker,tmos_close_1hour,date_hour_index,close_1hour_w1_roc,close_1hour_w1_diff,volume_1hour_w1_roc,volume_1hour_w1_diff,tmos_close_1hour_w1_roc,tmos_close_1hour_w1_diff,close_1hour_w5_ma,close_1hour_w5_std,close_1hour_w5_norm_std,close_1hour_w5_ma_low_2std,close_1hour_w5_ma_up_2std,close_1hour_w5_ma_low_3std,close_1hour_w5_ma_up_3std,close_1hour_w5_mean_abs_pct,close_1hour_w5_alpha,close_1hour_w5_min,close_1hour_w5_max,close_1hour_w5_rsi,...,close_1hour_w120_lvl_-0.99-0.995,close_1hour_w120_lvl_1.01-1.02,close_1hour_w120_lvl_-0.98-0.99,close_1hour_w120_lvl_1.02-1.03,close_1hour_w120_lvl_-0.97-0.98,close_1hour_w120_lvl_1.03-1.04,close_1hour_w120_lvl_-0.96-0.97,close_1hour_w120_lvl_1.04-1.05,close_1hour_w120_lvl_-0.95-0.96,close_1hour_w120_lvl_1.05-1.07,close_1hour_w120_lvl_-0.9299999999999999-0.95,tmos_close_1hour_w120_lvl_1-1.005,tmos_close_1hour_w120_lvl_-0.995-1,tmos_close_1hour_w120_lvl_1.005-1.01,tmos_close_1hour_w120_lvl_-0.99-0.995,tmos_close_1hour_w120_lvl_1.01-1.015,tmos_close_1hour_w120_lvl_-0.985-0.99,tmos_close_1hour_w120_lvl_1.015-1.02,tmos_close_1hour_w120_lvl_-0.98-0.985,tmos_close_1hour_w120_lvl_1.02-1.03,tmos_close_1hour_w120_lvl_-0.97-0.98,tmos_close_1hour_w120_lvl_1.03-1.04,tmos_close_1hour_w120_lvl_-0.96-0.97,tmos_close_1hour_w120_lvl_1.04-1.05,tmos_close_1hour_w120_lvl_-0.95-0.96
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
212213,212213,2025-02-04 18:00:00,4021.5,12373.0,YDEX,6.38,2025-02-04 18:00:00,-0.000373,-1.5,-0.675198,-25721.0,0.000000,0.00,4033.8,10.837435,0.002687,4012.125130,4055.474870,4001.287695,4066.312305,0.001312,-0.137292,4021.5,4045.5,9.433962,...,6.0,26.0,12.0,19.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,5.0,31.0,10.0,21.0,1.0,15.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
212214,212214,2025-02-04 19:00:00,4019.0,8899.0,YDEX,6.40,2025-02-04 19:00:00,-0.000622,-2.5,-0.280773,-3474.0,0.003135,0.02,4028.5,10.142731,0.002518,4008.214537,4048.785463,3998.071806,4058.928194,0.001312,-0.143378,4019.0,4039.5,0.000000,...,4.0,23.0,12.0,22.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,14.0,27.0,3.0,17.0,9.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212332,212332,2025-02-17 14:00:00,4654.5,41682.0,YDEX,7.15,2025-02-17 14:00:00,0.001075,5.0,-0.653769,-78706.0,-0.002789,-0.02,4641.9,12.017695,0.002589,4617.864609,4665.935391,4605.846914,4677.953086,0.006399,0.065778,4626.5,4654.5,86.006826,...,2.0,0.0,4.0,0.0,13.0,0.0,8.0,0.0,2.0,0.0,7.0,5.0,1.0,0.0,0.0,0.0,2.0,0.0,10.0,0.0,11.0,0.0,3.0,0.0,0.0
212333,212333,2025-02-17 15:00:00,4675.5,66360.0,YDEX,7.18,2025-02-17 15:00:00,0.004512,21.0,0.592054,24678.0,0.004196,0.03,4647.6,19.482043,0.004192,4608.635914,4686.564086,4589.153871,4706.046129,0.002993,0.079370,4626.5,4675.5,70.503597,...,4.0,0.0,3.0,0.0,10.0,0.0,12.0,0.0,3.0,0.0,6.0,0.0,7.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,19.0,0.0,3.0,0.0,1.0


In [94]:
#close, volume, tmos_close
df_dev_ticker['close'].values == df_1day_ticker['close_1hour'].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [96]:
pd.concat([df_dev_ticker['close'].reset_index(drop=True), df_1day_ticker['close_1hour'].reset_index(drop=True)], axis=1)

,close,close_1hour
0,4021.5,4021.5
1,4019.0,4019.0
...,...,...
119,4654.5,4654.5
120,4675.5,4675.5


In [98]:
df_dev_ticker['volume'].values == df_1day_ticker['volume_1hour'].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [100]:
df_dev_ticker['tmos_close'].values == df_1day_ticker['tmos_close_1hour'].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

### *15min

In [107]:
ticker = 'YDEX'
df_dev_ticker = load_pkl('../dev/data/preproc/15min/2025-02-17_16_15/YDEX.pkl')
df_dev_ticker

,time,close,volume,ticker,tmos_close
0,2025-02-13 10:45:00,4464.0,99206.0,YDEX,6.94
1,2025-02-13 11:00:00,4459.5,62811.0,YDEX,6.93
...,...,...,...,...,...
119,2025-02-17 15:45:00,4675.5,17379.0,YDEX,7.18
120,2025-02-17 16:00:00,4676.0,20490.0,YDEX,7.18


In [111]:
df_1day = load_pkl('../data/feat_engin/df_fe.pkl')
df_1day

,time,close,volume,ticker,tmos_close,index_1hour,index_1day,result,delta_time,income_rate,res_price,res_ind,close_w1_roc,close_w1_diff,volume_w1_roc,volume_w1_diff,tmos_close_w1_roc,tmos_close_w1_diff,close_w5_ma,close_w5_std,close_w5_norm_std,close_w5_ma_low_2std,close_w5_ma_up_2std,close_w5_ma_low_3std,close_w5_ma_up_3std,...,close_w120_lvl_-0.99-0.995,close_w120_lvl_1.01-1.02,close_w120_lvl_-0.98-0.99,close_w120_lvl_1.02-1.03,close_w120_lvl_-0.97-0.98,close_w120_lvl_1.03-1.04,close_w120_lvl_-0.96-0.97,close_w120_lvl_1.04-1.05,close_w120_lvl_-0.95-0.96,close_w120_lvl_1.05-1.07,close_w120_lvl_-0.9299999999999999-0.95,tmos_close_w120_lvl_1-1.005,tmos_close_w120_lvl_-0.995-1,tmos_close_w120_lvl_1.005-1.01,tmos_close_w120_lvl_-0.99-0.995,tmos_close_w120_lvl_1.01-1.015,tmos_close_w120_lvl_-0.985-0.99,tmos_close_w120_lvl_1.015-1.02,tmos_close_w120_lvl_-0.98-0.985,tmos_close_w120_lvl_1.02-1.03,tmos_close_w120_lvl_-0.97-0.98,tmos_close_w120_lvl_1.03-1.04,tmos_close_w120_lvl_-0.96-0.97,tmos_close_w120_lvl_1.04-1.05,tmos_close_w120_lvl_-0.95-0.96
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2022-05-04 10:00:00,31.52,165423.0,AFLT,4.30,NaN,NaN,LOSE,0 days 00:15:00,0.994000,31.92,1,NaN,NaN,NaN,NaN,NaN,NaN,31.520,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-05-04 10:15:00,31.92,99825.0,AFLT,4.32,NaN,NaN,LOSE,0 days 00:15:00,0.994000,32.40,2,0.012690,0.40,-0.396547,-65598.0,0.004651,0.02,31.720,0.282843,0.008917,31.154315,32.285685,30.871472,32.568528,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2362115,2025-02-17 23:30:00,396.48,2099.0,BSPB,7.33,594230.0,46868.0,DNF,0 days 00:15:00,0.996276,397.56,2362116,-0.002616,-1.04,1.694480,1320.0,0.002736,0.02,396.906,0.557835,0.001405,395.790330,398.021670,395.232495,398.579505,...,14.0,0.0,23.0,0.0,7.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,13.0,0.0,9.0,0.0,40.0,0.0,8.0,0.0,38.0
2362116,2025-02-17 23:45:00,397.56,1407.0,BSPB,7.34,594231.0,46869.0,DNF,0 days 00:00:00,0.999000,397.56,2362116,0.002724,1.08,-0.329681,-692.0,0.001364,0.01,396.966,0.618288,0.001558,395.729424,398.202576,395.111136,398.820864,...,33.0,0.0,24.0,0.0,9.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,14.0,0.0,6.0,0.0,43.0,0.0,8.0,0.0,33.0


In [117]:
mask_time = (pd.Timestamp('2025-02-13 10:45') <= df_1day['time']) & (df_1day['time'] <= pd.Timestamp('2025-02-17 16:00'))
mask = (df_1day['ticker'] == ticker) & mask_time
df_1day_ticker = df_1day[mask]
df_1day_ticker

,time,close,volume,ticker,tmos_close,index_1hour,index_1day,result,delta_time,income_rate,res_price,res_ind,close_w1_roc,close_w1_diff,volume_w1_roc,volume_w1_diff,tmos_close_w1_roc,tmos_close_w1_diff,close_w5_ma,close_w5_std,close_w5_norm_std,close_w5_ma_low_2std,close_w5_ma_up_2std,close_w5_ma_low_3std,close_w5_ma_up_3std,...,close_w120_lvl_-0.99-0.995,close_w120_lvl_1.01-1.02,close_w120_lvl_-0.98-0.99,close_w120_lvl_1.02-1.03,close_w120_lvl_-0.97-0.98,close_w120_lvl_1.03-1.04,close_w120_lvl_-0.96-0.97,close_w120_lvl_1.04-1.05,close_w120_lvl_-0.95-0.96,close_w120_lvl_1.05-1.07,close_w120_lvl_-0.9299999999999999-0.95,tmos_close_w120_lvl_1-1.005,tmos_close_w120_lvl_-0.995-1,tmos_close_w120_lvl_1.005-1.01,tmos_close_w120_lvl_-0.99-0.995,tmos_close_w120_lvl_1.01-1.015,tmos_close_w120_lvl_-0.985-0.99,tmos_close_w120_lvl_1.015-1.02,tmos_close_w120_lvl_-0.98-0.985,tmos_close_w120_lvl_1.02-1.03,tmos_close_w120_lvl_-0.97-0.98,tmos_close_w120_lvl_1.03-1.04,tmos_close_w120_lvl_-0.96-0.97,tmos_close_w120_lvl_1.04-1.05,tmos_close_w120_lvl_-0.95-0.96
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
845407,2025-02-13 10:45:00,4464.0,99206.0,YDEX,6.94,212303.0,16685.0,LOSE,0 days 00:45:00,0.994,4499.5,845410,0.000560,2.5,0.193513,16085.0,0.001443,0.01,4480.6,24.983495,0.005576,4430.633011,4530.566989,4405.649516,4555.550484,...,2.0,8.0,2.0,1.0,26.0,0.0,24.0,0.0,41.0,0.0,9.0,2.0,4.0,2.0,3.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,26.0,0.0,23.0
845408,2025-02-13 11:00:00,4459.5,62811.0,YDEX,6.93,212303.0,16685.0,LOSE,0 days 00:30:00,0.994,4499.5,845410,-0.001008,-4.5,-0.366863,-36395.0,-0.001441,-0.01,4468.6,13.310710,0.002979,4441.978580,4495.221420,4428.667870,4508.532130,...,1.0,8.0,3.0,2.0,26.0,0.0,31.0,0.0,36.0,0.0,5.0,3.0,3.0,3.0,2.0,2.0,1.0,3.0,0.0,0.0,1.0,0.0,31.0,0.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845526,2025-02-17 15:45:00,4675.5,17379.0,YDEX,7.18,212333.0,16687.0,LOSE,0 days 02:00:00,0.994,4702.5,845534,0.002681,12.5,0.259896,3585.0,0.001395,0.01,4663.5,7.897784,0.001694,4647.704431,4679.295569,4639.806647,4687.193353,...,16.0,0.0,12.0,0.0,39.0,0.0,41.0,0.0,5.0,0.0,0.0,1.0,27.0,0.0,8.0,0.0,5.0,0.0,6.0,0.0,63.0,0.0,7.0,0.0,0.0
845527,2025-02-17 16:00:00,4676.0,20490.0,YDEX,7.18,212333.0,16687.0,LOSE,0 days 01:45:00,0.994,4702.5,845534,0.000107,0.5,0.179009,3111.0,0.000000,0.00,4667.8,7.620696,0.001633,4652.558609,4683.041391,4644.937913,4690.662087,...,15.0,0.0,13.0,0.0,39.0,0.0,39.0,0.0,6.0,0.0,0.0,1.0,27.0,0.0,8.0,0.0,5.0,0.0,6.0,0.0,63.0,0.0,6.0,0.0,0.0


In [119]:
#close, volume, tmos_close
df_dev_ticker['close'].values == df_1day_ticker['close'].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [ ]:
'close/close_w30_expma'

In [121]:
pd.concat([df_dev_ticker['close'].reset_index(drop=True), df_1day_ticker['close'].reset_index(drop=True)], axis=1)

,close,close
0,4464.0,4464.0
1,4459.5,4459.5
...,...,...
119,4675.5,4675.5
120,4676.0,4676.0


In [123]:
df_dev_ticker['volume'].values == df_1day_ticker['volume'].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [125]:
df_dev_ticker['tmos_close'].values == df_1day_ticker['tmos_close'].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [ ]:
# Если совпадают close, то как 1 фича может отличаться хоть и на чуть-чуть